# 🧮 AAPL Stock Forecasting with ARIMA and LSTM

This notebook demonstrates time series forecasting for AAPL using:
- ARIMA (classical statistical method)
- LSTM (deep learning method)

**Note:** Close price is used for modeling.

In [ ]:
# ✅ Install dependencies (for Colab)
!pip install yfinance pmdarima tensorflow

In [ ]:
# ✅ Imports
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## 📥 Load AAPL Close Price Data

In [ ]:
data = yf.download("AAPL", start="2023-01-01", end="2024-12-31")['Close']
data = data.asfreq('B')  # business days
data = data.fillna(method='ffill')
data.plot(title='AAPL Close Price', figsize=(12,4))
plt.grid(True)
plt.show()

## 🔁 ARIMA Model

In [ ]:
# Fit ARIMA with auto_arima
model = pm.auto_arima(data, seasonal=False, stepwise=True, suppress_warnings=True)
model.summary()

In [ ]:
# Forecast next 30 business days
n_periods = 30
forecast, conf_int = model.predict(n_periods=n_periods, return_conf_int=True)

# Plot
index_of_fc = pd.date_range(data.index[-1], periods=n_periods+1, freq='B')[1:]
fc_series = pd.Series(forecast, index=index_of_fc)

plt.figure(figsize=(12, 5))
plt.plot(data, label='Historical')
plt.plot(fc_series, label='Forecast')
plt.fill_between(index_of_fc, conf_int[:, 0], conf_int[:, 1], color='pink', alpha=0.3)
plt.title('ARIMA Forecast - AAPL')
plt.grid(True)
plt.legend()
plt.show()

## 🔮 LSTM Model (Deep Learning)

In [ ]:
# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(np.array(data).reshape(-1,1))

# Prepare sequences
X = []
y = []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i])
    y.append(scaled_data[i])
X, y = np.array(X), np.array(y)

In [ ]:
# Build LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
model_lstm.add(LSTM(50))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Train
model_lstm.fit(X, y, epochs=5, batch_size=32, verbose=1)

In [ ]:
# Predict last 30 points
inputs = scaled_data[-60:]
temp_input = list(inputs)

lstm_predictions = []
for i in range(30):
    x_input = np.array(temp_input[-60:]).reshape(1, 60, 1)
    pred = model_lstm.predict(x_input, verbose=0)
    temp_input.append(pred[0])
    lstm_predictions.append(pred[0])

# Inverse scale and plot
lstm_predictions = scaler.inverse_transform(lstm_predictions)
future_dates = pd.date_range(data.index[-1], periods=31, freq='B')[1:]

plt.figure(figsize=(12, 5))
plt.plot(data, label='Historical')
plt.plot(future_dates, lstm_predictions, label='LSTM Forecast')
plt.title('LSTM Forecast - AAPL')
plt.grid(True)
plt.legend()
plt.show()